# WEAP desde simgen

In [1]:
import win32com.client #Libreria para poder usar win32com
import numpy as np
import glob, os
import pandas as pd
import matplotlib.pyplot as plt
import re
#funcion que elimina duplicados
def remove_duplicates(l):
    return list(set(l))
def existedir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
''' NOTA, LAS ESTACIONES DEBEN ESTAR EN EL SIGUIENTE ORDEN
"caren.csv_formateado.csv" 2000
"cogoti18.csv_formateado.csv" 2001
"eltome.csv_formateado.csv" 2002
"lasramadas.csv_formateado.csv" 2003
"OvalleDGA.csv_formateado.csv" 2004
"pabellon.csv_formateado.csv" 2005
"palomaembalse.csv_formateado.csv" 2006
"punitaqui.csv_formateado.csv" 2007
"rapel.csv_formateado.csv" 2008
"recoletaembalse.csv_formateado.csv" 2009
"tascadero.csv_formateado.csv" 2010
"caren.csv_formateado.csv" 2000
"cogoti18.csv_formateado.csv" 2001
"eltome.csv_formateado.csv" 2002
"lasramadas.csv_formateado.csv" 2003
"OvalleDGA.csv_formateado.csv" 2004
"pabellon.csv_formateado.csv" 2005
"palomaembalse.csv_formateado.csv" 2006
"punitaqui.csv_formateado.csv" 2007
"rapel.csv_formateado.csv" 2008
"recoletaembalse.csv_formateado.csv" 2009
"tascadero.csv_formateado.csv" 2010
'''

' NOTA, LAS ESTACIONES DEBEN ESTAR EN EL SIGUIENTE ORDEN\n"caren.csv_formateado.csv" 2000\n"cogoti18.csv_formateado.csv" 2001\n"eltome.csv_formateado.csv" 2002\n"lasramadas.csv_formateado.csv" 2003\n"OvalleDGA.csv_formateado.csv" 2004\n"pabellon.csv_formateado.csv" 2005\n"palomaembalse.csv_formateado.csv" 2006\n"punitaqui.csv_formateado.csv" 2007\n"rapel.csv_formateado.csv" 2008\n"recoletaembalse.csv_formateado.csv" 2009\n"tascadero.csv_formateado.csv" 2010\n"caren.csv_formateado.csv" 2000\n"cogoti18.csv_formateado.csv" 2001\n"eltome.csv_formateado.csv" 2002\n"lasramadas.csv_formateado.csv" 2003\n"OvalleDGA.csv_formateado.csv" 2004\n"pabellon.csv_formateado.csv" 2005\n"palomaembalse.csv_formateado.csv" 2006\n"punitaqui.csv_formateado.csv" 2007\n"rapel.csv_formateado.csv" 2008\n"recoletaembalse.csv_formateado.csv" 2009\n"tascadero.csv_formateado.csv" 2010\n'

In [5]:
###config###
#path: directorio con archivos en formato para weap, sin / al final
#cambiar 25 por el percentil que se desee calcular
#pathdatosWEAP: directorio de datos de WEAP, sin / al final
path = "C:/Users/Pablo/Dropbox/cazalac/Simgen-Limari v2/simgen/output_sim/mensual/"
#path = "C:/Users/Pablo/Dropbox/cazalac/simgen_v3/output_sim/mensual/"
percentiles = ['10','50','95']
#percentiles = ['1','2','3','4','5','8','10','15','25','50','75','85','90','96','97','98','99']
pathdatosWEAP = "C:/Users/Pablo/Documents/WEAP Areas/Cuenca del Limari_V1/Datos"
salidaweap ="C:/Users/Pablo/Desktop/WEAP/salida" #donde se guardarán los resultados de weap
#abrir weap
WEAP=win32com.client.Dispatch("WEAP.WEAPApplication")
#modelo
WEAP.ActiveArea = "Cuenca del Limari_V1"
#inicio de archivo de weap
ListSeparator='$ListSeparator=,'

In [12]:
for nn in range(0,len(percentiles)):
    print nn
    existedir(path)
    existedir(path+percentiles[nn])
    print path+percentiles[nn]
    #generamos una nueva lista
    simt=[]
    fix=1 #para generar el index con el primer archivo
    #usamos las pp para generar el index
    print glob.glob("*")
    print glob.glob("pp_*.csv")
    for file in glob.glob("pp_*.csv"):
        simt.append(file[-10:-4])
    sim = remove_duplicates(simt)
    sim = [s.replace('pp_', '') for s in sim]
    sim = [s.replace('p_', '') for s in sim]
    for i in range(0,len(sim)):
        print "Trabajando en sim: "+str(sim[i])
        pp = pd.read_csv("pp_"+str(sim[i])+".csv", sep=",", index_col='Unnamed: 0')
        tt = pd.read_csv("tprom_"+str(sim[i])+".csv", sep=",", index_col='Unnamed: 0')
        #archivo de pp
        precipitacionesfile = open(pathdatosWEAP+'/'+'PrecCuencasAltasb.csv', 'w')
        tempfile = open(pathdatosWEAP+'/'+'TempCuencasAltas.csv', 'w')
        #generamos el formato weap
        precipitaciones = [str(ListSeparator)]
        temperatura = []
        #para precipitaciones
        for k in range(0,len(pp)):
            year = pd.DatetimeIndex(pp.iloc[[k]].index).year
            month = pd.DatetimeIndex(pp.iloc[[k]].index).month
            #CUIDADO: se debe modificar por el orden de las estaciones, estaciones modelo weap limari
            '''|  año  |  mes  |    2005  | 0 | 0 |     2003    |     2010  |||     2001  | 0 |        2009      |      2006      |||   2002  |    2007   |     2004   ||'''
            precipitaciones.append(','.join([str(year[0]),str(month[0]),str(pp['2005'][k]),str(0),str(0),str(pp['2003'][k]),str(pp['2010'][k]),str(pp['2001'][k]),str(0),str(pp['2009'][k]),str(pp['2006'][k]),str(pp['2002'][k]),str(pp['2007'][k]),str(pp['2004'][k])]))
        for item in precipitaciones:
            precipitacionesfile.write("%s\n" % item)
        precipitacionesfile.close()
        #para temperatura, se hacen de forma separada porque la temperatura empieza en 1970
        #generamos una mascara para usar informacion desde 1970-05 como en archivo TempCuencasAltas
        mask = (tt.index > "1970-04-31")
        tt2 = tt[mask]
        for j in range(0,len(tt2)):
            year = pd.DatetimeIndex(tt2.iloc[[j]].index).year
            month = pd.DatetimeIndex(tt2.iloc[[j]].index).month
            #CUIDADO: se debe modificar por el orden de las estaciones
            temperatura.append(','.join([str(year[0]),str(month[0]),str(tt2['2003'][j]),str(tt2['2006'][j])]))
        for item in temperatura:
            tempfile.write("%s\n" % item)
        tempfile.close()
        ########modificar WEAP.EndYear=2080 por valor necesario##############################################
        #forzamos WEAP 2080:
        WEAP.EndYear=2080
        #volvemos a forzar 2080 con el calculo -Verificar en modelo-
        WEAP.Calculate(2080) #Empieza a calcular
        SALIDA = salidaweap+"/"+percentiles[nn]+"/"+str(sim[i])+"Resultado.csv"
        WEAP.ExportResults(SALIDA,False)

0
C:/Users/Pablo/Dropbox/cazalac/Simgen-Limari v2/simgen/output_sim/mensual/10
['graficov2.R', 'promedios_futuros_simgen.R', 'R', 'resumen', 'resumen-salida-simgen.ipynb', 'Untitled1.ipynb', 'WEAP (2).ipynb', 'weap-malla.ipynb', 'WEAP.ipynb']
[]
1
C:/Users/Pablo/Dropbox/cazalac/Simgen-Limari v2/simgen/output_sim/mensual/50
['graficov2.R', 'promedios_futuros_simgen.R', 'R', 'resumen', 'resumen-salida-simgen.ipynb', 'Untitled1.ipynb', 'WEAP (2).ipynb', 'weap-malla.ipynb', 'WEAP.ipynb']
[]
2
C:/Users/Pablo/Dropbox/cazalac/Simgen-Limari v2/simgen/output_sim/mensual/95
['graficov2.R', 'promedios_futuros_simgen.R', 'R', 'resumen', 'resumen-salida-simgen.ipynb', 'Untitled1.ipynb', 'WEAP (2).ipynb', 'weap-malla.ipynb', 'WEAP.ipynb']
[]


In [55]:
#procesamiento de salida WEAP
os.chdir(salidaweap)
simt = []
for file in glob.glob("*Resultado.csv"):
    simt.append(file[0:-13])
sim = remove_duplicates(simt)
sim = [s.replace('pp_', '') for s in sim]
sim = [s.replace('p_', '') for s in sim]
print sim

[]


In [54]:
#generar resultados
resultado = pd.read_csv(str(sim[0])+'Resultado.csv', sep=",", skiprows=1, header=None)
resultado = resultado.T
resultado = resultado.rename(columns={0: 'Embalse-Cogoti', 1: 'Embalse-Paloma', 2: 'Embalse-Recoleta'})
resultado = resultado[1:]
del resultado[3]
resultado.index = pd.date_range('05/01/1970', periods=len(resultado), freq='M')
print sim[0]
resultado.astype(int).resample('A').mean().mean()/1000000

IndexError: list index out of range